In [30]:
import tensorflow as tf
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('Cleaned_Apparel.csv')

# Map categorical columns to integer labels
gender_map = {'Men': 0, 'Women': 1}
season_map = {'Fall': 0, 'Summer': 1, 'Winter': 2, 'Spring': 3}
emotion_map = {'Neutral': 0, 'Contentment': 1, 'Sadness': 2, 'Anger': 3, 'Fear': 4, 'Joy': 5}

# Apply the mappings to the columns
df['gender'] = df['gender'].map(gender_map)
df['season'] = df['season'].map(season_map)
df['Emotion_Category'] = df['Emotion_Category'].map(emotion_map)

# Encode categorical columns using tf.keras.layers.CategoryEncoding
gender_encoder = tf.keras.layers.CategoryEncoding(num_tokens=2, output_mode='multi_hot', sparse=False)
season_encoder = tf.keras.layers.CategoryEncoding(num_tokens=4, output_mode='multi_hot', sparse=False)
emotion_encoder = tf.keras.layers.CategoryEncoding(num_tokens=6, output_mode='multi_hot', sparse=False)

# Apply CategoryEncoding to each column
encoded_gender = gender_encoder(df['gender'].values)
encoded_season = season_encoder(df['season'].values)
encoded_emotion = emotion_encoder(df['Emotion_Category'].values)

# Combine the encoded features into a single tensor
features = tf.concat([encoded_gender, encoded_season, encoded_emotion], axis=-1)

# Check the shape of the features
print(f"Features shape: {features.shape}")

# Split the data into training (70%) and testing (30%) using TensorFlow slicing
dataset_size = len(df)
train_size = int(0.7 * dataset_size)

train_features = features[:train_size]
test_features = features[train_size:]

train_labels = df['id_x'][:train_size]
test_labels = df['id_x'][train_size:]

# Print the shape of train_features to check if it has more than one dimension
print(f"train_features shape: {train_features.shape}")

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(train_features.shape[1],)),  # Ensure train_features has more than one dimension
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer to predict id_x
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(train_features, train_labels, epochs=10, batch_size=2)

# Evaluate the model on the test set
test_loss = model.evaluate(test_features, test_labels)
print(f"Test Loss: {test_loss}")

# Predicting using the model
predictions = model.predict(test_features)
print(predictions)


Features shape: (12,)
train_features shape: (12,)


IndexError: tuple index out of range